# 6. support vector machines
给定训练集样本 $D=\{(x_1,y_1),(x_2,y_2),...(x_m,y_m)\}, y\in\{-1,+1\}$,分类学习最基本的想法就是基于训练集D在样本空间中找到一个划分超平面，将不同类别的样本分开。由于数据点都在二维平面上，所以此时分割超平面就是一个平面。更高维的情况可以以此类推。如果数据集是三维的，那么此时用来分割数据的就是一个平面。超平面也就是分类的决策边界，分布在超平面一侧的所有数据都数以某个分类，另一侧的数据则属于另一个类别。

如果数据点离决策越远，那么其最后的预测结果也就越可信。我们希望找到离分割超平面最近的点，确保他们离分隔超平面的距离尽可能的大。这里点到分隔面的距离被称为间隔。

支持向量（support vector）就是离分隔超平面最近的那些点。

在样本空间中，分隔超平面可以用如下的线性方程来描述$w^Tx+b=0$，其中$w = (w_1;w_2....;w_d)$为法向量，决定了超平面的方向；b为位移项，决定了超平面与原点之间的距离。显然超平面可以被法向量w和位移b确定，将其记为$(w,b)$，样本空间中任一点x到超平面$(w,b)$的距离可写为$r = \frac{||w^TA+b||}{||w||}$。

假设超平面$(w,b)$能将训练样本正确分类，即对$(x_i,y_i)\in D$若$y_i=+1$，则有

$\begin{cases}
w^Tx_i+b\ge+1, y_i=+1\\
w^Tx_i+b\le-1,y_i=-1
\end{cases}
$

![svm.png](svm.png)

如图所示，距离超平面最近的这几个训练样本点使等号成立，他们被称为“支持向量”两个异类支持向量到超平面的距离之和为$r = \frac{2}{||w||}$，他被称为“间隔”。欲找到具有“最大间隔”(maximum margin)的划分超平面，也就是要找到能满足式中约束的参数$w$和$b$使得r最大，即

$
\max\limits_{w,1}\frac{2}{||w||} \\
s.t. \qquad y_i(w^Tx_i+b)\ge1, i = 1,2,...m
$

为了最大化间隔，仅需最大化$||w||^-1$，这个等价于最小化$||w||$于是可重写为：

$
\min\limits_{w,b} \frac{1}{2}||w||^2 \\
s.t. \qquad y_i(w^Tx_i+b)\ge1, i=1,2,...m
$

这就是支持向量机的基本型。

简化版的SMO算法：

platt SMO算法中的外驯化确定要优化的最佳alpha对。而简化版的却会跳过这一部分，首先在数据集上遍历每一个alpha，然后在剩下的alpha集合中随机选择另一个alpha，从而构架alpha对。这里有一点相当重要，就是要同时改变两个alpha。之所以这样做是因为有一个约束条件：$\sum a_i \cdot label^{(i)}=0$由于改变一个alpha可能会导致该约束条件失效，因此总是同时改变这两个alpha。

In [3]:
import random

'''从某个范围内随机选择一个整数
    i : alpha的下标
    m : 所有alpha的数目

'''
def selectJrand(i,m):
    j=i
    while (j==i):
        j = int(random.uniform(0,m))
    return j


# 在数值过大时对其进行调整，确保aj在high和low之间
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

SMO函数的伪代码大致如下：
```
创建一个alpha向量并将其初始化为0向量
当迭代次数小于最大迭代次数时（外循环）
    对数据集中的每个数据向量（内循环）：
    如果该数据向量可以被优化：
        随机选择另一个数据向量
        tong
```